In [3]:
# !rm dataKu.csv
# !ls -a

 .				      LICENSE
 ..				      README.md
 .git				     'house price'
 .ipynb_checkpoints		      latihan_polars.ipynb
'Data Historis Nikel Berjangka.csv'   latihan_polars2.ipynb


In [4]:
import polars as pl

dataKu_csv = pl.DataFrame({
  'nama': [
    'andi tirtawansyah',
    'yuli nurlaila sari',
    'muhammad varza',
    'bhenaya',
    'sazisyam',
    'athaya'
  ],
  'tanggal_lahir': [
    '21 Des 1975', #Dec
    '12 Jul 1977',
    '19 Mei 2004', #May
    '17 Aug 2005',
    '12 Sep 2009',
    '23 Jul 2018'
  ],
  'berat_badan': [ 82.5, 70.0 , 88.0, 92.8, 49.5, 32.5 ],
  'tinggi': [ 1.68, 1.64, 1.76, 1.71, 1.69, 1.45 ],
  'gender': [ 'male', 'female', 'male', 'male', 'male', 'male' ]
})
dataKu_csv.write_csv('dataKu.csv')

dataKu_csv

nama,tanggal_lahir,berat_badan,tinggi,gender
str,str,f64,f64,str
"""andi tirtawansyah""","""21 Des 1975""",82.5,1.68,"""male"""
"""yuli nurlaila sari""","""12 Jul 1977""",70.0,1.64,"""female"""
"""muhammad varza""","""19 Mei 2004""",88.0,1.76,"""male"""
"""bhenaya""","""17 Aug 2005""",92.8,1.71,"""male"""
"""sazisyam""","""12 Sep 2009""",49.5,1.69,"""male"""
"""athaya""","""23 Jul 2018""",32.5,1.45,"""male"""


In [5]:
import seaborn as sns
import polars.selectors as cs
import polars as pl

data = r'dataKu.csv'
df = pl.read_csv(data, try_parse_dates=True, n_threads=1, low_memory=True)

df

nama,tanggal_lahir,berat_badan,tinggi,gender
str,str,f64,f64,str
"""andi tirtawansyah""","""21 Des 1975""",82.5,1.68,"""male"""
"""yuli nurlaila sari""","""12 Jul 1977""",70.0,1.64,"""female"""
"""muhammad varza""","""19 Mei 2004""",88.0,1.76,"""male"""
"""bhenaya""","""17 Aug 2005""",92.8,1.71,"""male"""
"""sazisyam""","""12 Sep 2009""",49.5,1.69,"""male"""
"""athaya""","""23 Jul 2018""",32.5,1.45,"""male"""


In [6]:
result = df.with_columns(
  pl.col('tanggal_lahir')
  .str.replace_many({
    'Mei': 'May',
    'Des': 'Dec',
  })
  .str.to_date('%d %b %Y', strict=False, exact=False),
  pl.col('berat_badan', 'tinggi').cast(pl.Float32)
)

result

nama,tanggal_lahir,berat_badan,tinggi,gender
str,date,f32,f32,str
"""andi tirtawansyah""",1975-12-21,82.5,1.68,"""male"""
"""yuli nurlaila sari""",1977-07-12,70.0,1.64,"""female"""
"""muhammad varza""",2004-05-19,88.0,1.76,"""male"""
"""bhenaya""",2005-08-17,92.800003,1.71,"""male"""
"""sazisyam""",2009-09-12,49.5,1.69,"""male"""
"""athaya""",2018-07-23,32.5,1.45,"""male"""


In [7]:
result.with_columns(
  pl.when(pl.col('gender') == 'male')
  .then('king: ' + pl.col('nama'))
  .otherwise('queen: ' + pl.col('nama'))
  .alias('nama')
).select(
  pl.col('nama').alias('Name'),
  pl.col('tanggal_lahir').alias('Birth Date')
)

Name,Birth Date
str,date
"""king: andi tirtawansyah""",1975-12-21
"""queen: yuli nurlaila sari""",1977-07-12
"""king: muhammad varza""",2004-05-19
"""king: bhenaya""",2005-08-17
"""king: sazisyam""",2009-09-12
"""king: athaya""",2018-07-23


In [8]:
result.filter(
  pl.col('tanggal_lahir')
  .dt.year() > 2_000
).select(
  pl.col('nama').alias('Nama'),
  pl.col('tanggal_lahir').alias('Tanggal Lahir')
)

Nama,Tanggal Lahir
str,date
"""muhammad varza""",2004-05-19
"""bhenaya""",2005-08-17
"""sazisyam""",2009-09-12
"""athaya""",2018-07-23


In [9]:
result.group_by(
  (pl.col('tanggal_lahir').dt.year() // 100 * 100).alias('Tahun Lahir')
  .cast(pl.UInt16)
).agg(
  ((pl.col('berat_badan').mean() + pl.col('berat_badan').median()) / 2).alias('Berat Badan'),
  pl.len().alias('Jumlah')
).sort('Tahun Lahir')

Tahun Lahir,Berat Badan,Jumlah
u16,f32,u32
1900,76.25,2
2000,67.224998,4


In [10]:
result.with_columns(
  (pl.col('tanggal_lahir').dt.year() // 100 * 100).alias('tl_rounded')
  .cast(pl.UInt16)
).pivot('tl_rounded', index='gender', values='berat_badan', aggregate_function='mean')

gender,1900,2000
str,f32,f32
"""male""",82.5,65.699997
"""female""",70.0,null
